In [1]:
import numpy as np
import pandas as pd
import scipy
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import psycopg2
from sqlalchemy import create_engine, text

POSTGRES_ADDRESS = 'ds4ateam32.cl8xbcsviig5.us-west-2.rds.amazonaws.com'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'DS4ATEAM32'
POSTGRES_PASSWORD = 'DS4ATEAM32'
POSTGRES_DBNAME = 'DS4ATEAM32'
postgres_str = 'postgresql://' + POSTGRES_USERNAME + ':' + POSTGRES_PASSWORD + '@' + POSTGRES_ADDRESS + ':' + POSTGRES_PORT + '/' + POSTGRES_DBNAME
#print(postgres_str)
engine = create_engine(postgres_str)

q = """
SELECT * 
FROM cs_general2
;"""
df = pd.read_sql_query(q, engine)
df.head()

,cluster,edad,sexo,remuneracion_c_remunerado,remuneracion_c_no_remunerado,remuneracion_c_indeterminado,remuneracion_p_remunerado,remuneracion_p_no_remunerado,remuneracion_p_indeterminado,escolaridad_c_no_escolaridad,...,remuneracion_x_escolaridad_c_no_remunerado_x_basico,remuneracion_x_escolaridad_c_remunerado_x_avanzado,remuneracion_x_escolaridad_c_no_remunerado_x_avanzado,remuneracion_dado_escolaridad_p_remunerado_dado_no_escolaridad,remuneracion_dado_escolaridad_p_no_remunerado_dado_no_escolarid,remuneracion_dado_escolaridad_p_remunerado_dado_basico,remuneracion_dado_escolaridad_p_no_remunerado_dado_basico,remuneracion_dado_escolaridad_p_remunerado_dado_avanzado,remuneracion_dado_escolaridad_p_no_remunerado_dado_avanzado,poblacion_total
0,0,1,H,0,0,59903,0.00,0.00,1.00,0,...,0,0,0,0.00,0.00,0.00,0.00,0.0,0.0,59903
1,0,1,M,0,0,56630,0.00,0.00,1.00,0,...,0,0,0,0.00,0.00,0.00,0.00,0.0,0.0,56630
2,0,2,H,0,0,67939,0.00,0.00,1.00,1352,...,0,0,0,0.00,0.00,0.00,0.00,0.0,0.0,67939
3,0,2,M,0,0,65398,0.00,0.00,1.00,1120,...,0,0,0,0.00,0.00,0.00,0.00,0.0,0.0,65398
4,0,3,H,1518,74336,1676,0.02,0.96,0.02,500,...,73849,0,0,0.08,0.87,0.02,0.96,0.0,0.0,77530


In [3]:
df.columns

Index(['cluster', 'edad', 'sexo', 'remuneracion_c_remunerado',
       'remuneracion_c_no_remunerado', 'remuneracion_c_indeterminado',
       'remuneracion_p_remunerado', 'remuneracion_p_no_remunerado',
       'remuneracion_p_indeterminado', 'escolaridad_c_no_escolaridad',
       'escolaridad_c_basico', 'escolaridad_c_avanzado',
       'escolaridad_c_indeterminado', 'escolaridad_p_no_escolaridad',
       'escolaridad_p_basico', 'escolaridad_p_avanzado',
       'escolaridad_p_indeterminado', 'salud_c_atencion_formal',
       'salud_c_atencion_no_formal', 'salud_c_atencion_indeterminado',
       'salud_p_atencion_formal', 'salud_p_atencion_no_formal',
       'salud_p_atencion_indeterminado', 'pareja_c_si', 'pareja_c_no',
       'pareja_c_indeterminado', 'pareja_p_si', 'pareja_p_no',
       'pareja_p_indeterminado', 'hijos_c_0', 'hijos_c_1', 'hijos_c_2',
       'hijos_c_3_o_mas', 'hijos_p_0', 'hijos_p_1', 'hijos_p_2',
       'hijos_p_3_o_mas', 'inmigracion1_c_no', 'inmigracion1_c_si',
    

## Idea principal
Se busca tomar una infografía base e irla filtrando hasta el punto que teniendo en cuenta las mismas variables se denotan diferencias de sexo en la cantidad de personas.

## Escena 1
Municipios se dividen en clusters de manera que se confunda la variable socio-económica.

In [4]:
# Aquí irían los clusters, se dividieron entre cabeceras y no cabeceras en este caso
#iterator = df['cluster'].unique()
iterator = [3, 2]
iterator

[3, 2]

# Orden de clusters, 3 (alto) - 4 - 0 - 1 - 2 (bajo)

## Escena 2:
Nacen 2 parejas de niños (H/M) en municipios escogidos dentro de los clusters. Análisis de edad 1 y 2 (0 a 9) en columna "enfermo", se espera que el valor sea distinto a "Nada". (Edad 1 y 2 no poseen información de trabajo)

No tiene significancia alguna, no se va a mostrar en la ppt. Se sugiere que en el proximo censo se tenga mas datos para educación

In [5]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'].isin([1, 2])) & (df['cluster'] == cluster)].copy()
    df_slice['tot_enfermos'] = df_slice['salud_c_atencion_formal'] + df_slice['salud_c_atencion_no_formal']
    df_slice_H = df_slice[df_slice['sexo'] == 'H']
    df_slice_M = df_slice[df_slice['sexo'] == 'M']
    porcentaje_atend_H = df_slice_H['salud_c_atencion_formal'].sum() / df_slice_H['tot_enfermos'].sum()
    porcentaje_atend_M = df_slice_M['salud_c_atencion_formal'].sum() / df_slice_M['tot_enfermos'].sum()
    print('La cantidad de niños edades 0-9 que reciben atención médica es {0:d}~{1:.2f}%'.
          format(df_slice_H['salud_c_atencion_formal'].sum(), porcentaje_atend_H * 100))
    print('La cantidad de niñas edades 0-9 que reciben atención médica es {0:d}~{1:.2f}%'.
          format(df_slice_M['salud_c_atencion_formal'].sum(), porcentaje_atend_M * 100))
    # Estadística
    crosstab_counts = df_slice.groupby('sexo')[['salud_c_atencion_formal', 'salud_c_atencion_no_formal']].sum()
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Atención médica formal {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 3
La cantidad de niños edades 0-9 que reciben atención médica es 134658~86.29%
La cantidad de niñas edades 0-9 que reciben atención médica es 121602~85.49%
Atención médica formal no es independiente de sexo

Para el cluster 2
La cantidad de niños edades 0-9 que reciben atención médica es 5751~79.34%
La cantidad de niñas edades 0-9 que reciben atención médica es 5356~78.60%
Atención médica formal es independiente de sexo



## Escena 3: 
Edad 3 (10 a 14) se analiza columna "trabajo", se espera tenga el valor de "Estudio"

In [6]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'] == 3) & (df['cluster'] == cluster)].copy()
    p_no_esc_H = df_slice[df_slice['sexo'] == 'H']['escolaridad_c_basico'] / df_slice[df_slice['sexo'] == 'H']['poblacion_total']
    p_no_esc_M = df_slice[df_slice['sexo'] == 'M']['escolaridad_c_basico'] / df_slice[df_slice['sexo'] == 'M']['poblacion_total']
    print('La cantidad de jovenes hombres edad 3 no escolarizados es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'H']['escolaridad_c_basico'].values[0], p_no_esc_H.values[0] * 100))
    print('La cantidad de jovenes mujeres edad 3 no escolarizados es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'M']['escolaridad_c_basico'].values[0], p_no_esc_M.values[0] * 100))
    # Estadística
    crosstab_counts = df_slice.groupby('sexo')[['escolaridad_c_basico', 'escolaridad_c_no_escolaridad']].sum()
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Estudio {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 3
La cantidad de jovenes hombres edad 3 no escolarizados es 870979~98.26%
La cantidad de jovenes mujeres edad 3 no escolarizados es 837114~98.45%
Estudio no es independiente de sexo

Para el cluster 2
La cantidad de jovenes hombres edad 3 no escolarizados es 55843~91.76%
La cantidad de jovenes mujeres edad 3 no escolarizados es 53836~92.18%
Estudio es independiente de sexo



## Escena 4:
Edad 3 (10 a 14) se espera que no tengan hijos.

Query: Obtener datos para 0, 1, 2 y 3+ hijos junto a variables de escolarizacion para compararlos

In [7]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'] == 3) & (df['cluster'] == cluster) & (df['sexo'] == 'M')].copy()
    print('La cantidad de jovenes edad 3 sin hijos es {0:d}~{1:.2f}%'.
          format(df_slice['hijos_c_0'].values[0], (df_slice['hijos_c_0'] / df_slice['poblacion_total'] * 100).values[0]))
    print('La cantidad de jovenes edad 3 con 1 hijo es {0:d}~{1:.2f}%'.
          format(df_slice['hijos_c_1'].values[0], (df_slice['hijos_c_1'] / df_slice['poblacion_total'] * 100).values[0]))
    print('La cantidad de jovenes edad 3 con 2 hijos es {0:d}~{1:.2f}%'.
          format(df_slice['hijos_c_2'].values[0], (df_slice['hijos_c_2'] / df_slice['poblacion_total'] * 100).values[0]))
    print('La cantidad de jovenes edad 3 con 3 o mas hijos es {0:d}~{1:.2f}%\n'.
          format(df_slice['hijos_c_3_o_mas'].values[0], (df_slice['hijos_c_3_o_mas'] / df_slice['poblacion_total'] * 100).values[0]))

Para el cluster 3
La cantidad de jovenes edad 3 sin hijos es 848383~99.77%
La cantidad de jovenes edad 3 con 1 hijo es 1407~0.17%
La cantidad de jovenes edad 3 con 2 hijos es 333~0.04%
La cantidad de jovenes edad 3 con 3 o mas hijos es 206~0.02%

Para el cluster 2
La cantidad de jovenes edad 3 sin hijos es 57696~98.79%
La cantidad de jovenes edad 3 con 1 hijo es 363~0.62%
La cantidad de jovenes edad 3 con 2 hijos es 122~0.21%
La cantidad de jovenes edad 3 con 3 o mas hijos es 221~0.38%



In [8]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'] == 3) & (df['cluster'] == cluster) & (df['sexo'] == 'M')].copy()
    p_nox0 = df_slice['escolaridad_x_hijos_c_no_escolaridad_x_0'] / df_slice[['escolaridad_x_hijos_c_no_escolaridad_x_0', 'escolaridad_x_hijos_c_basico_x_0']].sum(axis=1)
    p_nox1 = df_slice['escolaridad_x_hijos_c_no_escolaridad_x_1'] / df_slice[['escolaridad_x_hijos_c_no_escolaridad_x_1', 'escolaridad_x_hijos_c_basico_x_1']].sum(axis=1)
    p_nox2 = df_slice['escolaridad_x_hijos_c_no_escolaridad_x_2'] / df_slice[['escolaridad_x_hijos_c_no_escolaridad_x_2', 'escolaridad_x_hijos_c_basico_x_2']].sum(axis=1)
    p_nox3 = df_slice['escolaridad_x_hijos_c_no_escolaridad_x_3_o_mas'] / df_slice[['escolaridad_x_hijos_c_no_escolaridad_x_3_o_mas', 'escolaridad_x_hijos_c_basico_x_3_o_mas']].sum(axis=1)
    print('La cantidad de jovenes edad 3 sin hijos y no escolarizado es {0:d}~{1:.2f}%'.
          format(df_slice['escolaridad_x_hijos_c_no_escolaridad_x_0'].values[0], p_nox0.values[0] * 100))
    print('La cantidad de jovenes edad 3 sin hijos y no escolarizado es {0:d}~{1:.2f}%'.
          format(df_slice['escolaridad_x_hijos_c_no_escolaridad_x_1'].values[0], p_nox1.values[0] * 100))
    print('La cantidad de jovenes edad 3 sin hijos y no escolarizado es {0:d}~{1:.2f}%'.
          format(df_slice['escolaridad_x_hijos_c_no_escolaridad_x_2'].values[0], p_nox2.values[0] * 100))
    print('La cantidad de jovenes edad 3 sin hijos y no escolarizado es {0:d}~{1:.2f}%'.
          format(df_slice['escolaridad_x_hijos_c_no_escolaridad_x_3_o_mas'].values[0], p_nox3.values[0] * 100))
    # Estadística
    crosstab_counts = pd.DataFrame(data={'0_hijo':[df_slice['escolaridad_x_hijos_c_no_escolaridad_x_0'].values[0], 
                                               df_slice['escolaridad_x_hijos_c_basico_x_0'].values[0]],
                                    '1_hijo':[df_slice['escolaridad_x_hijos_c_no_escolaridad_x_1'].values[0],
                                             df_slice['escolaridad_x_hijos_c_basico_x_1'].values[0]],
                                    '2_hijo':[df_slice['escolaridad_x_hijos_c_no_escolaridad_x_2'].values[0],
                                             df_slice['escolaridad_x_hijos_c_basico_x_2'].values[0]],
                                    '3m_hijo':[df_slice['escolaridad_x_hijos_c_no_escolaridad_x_3_o_mas'].values[0],
                                              df_slice['escolaridad_x_hijos_c_basico_x_3_o_mas'].values[0]],
                                    'escolaridad':['no', 'basico']}).groupby('escolaridad').sum()
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Cantidad de hijos {0:s}es independiente de escolaridad\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 3
La cantidad de jovenes edad 3 sin hijos y no escolarizado es 3552~0.42%
La cantidad de jovenes edad 3 sin hijos y no escolarizado es 16~1.14%
La cantidad de jovenes edad 3 sin hijos y no escolarizado es 6~1.82%
La cantidad de jovenes edad 3 sin hijos y no escolarizado es 2~0.97%
Cantidad de hijos no es independiente de escolaridad

Para el cluster 2
La cantidad de jovenes edad 3 sin hijos y no escolarizado es 2774~4.95%
La cantidad de jovenes edad 3 sin hijos y no escolarizado es 45~13.39%
La cantidad de jovenes edad 3 sin hijos y no escolarizado es 12~10.53%
La cantidad de jovenes edad 3 sin hijos y no escolarizado es 5~2.30%
Cantidad de hijos no es independiente de escolaridad



## Escena 5: (Falta en tabla quantity)
Edad 4 (15 a 19) los jovenes que ya mantienen su hogar deberían ser alfabetas para poder trabajar y mantenerse dignamente, se tiene encuenta "parentesco", se espera que si tiene el valor "Jefe_Hogar" o "Pareja" entonces "alfabeta" sea "True". ¿Se filtra población?

Tabla: Falta cantidad de parentescos ("Jefe_Hogar" o "Pareja") en tabla quantity_by_cluster

## Escena 6
Edad 5 (20 a 39) luego de haber logrado el camino de la educación, de los no hijos y de la salud, ahora se evalúan las condiciones de vida. Empezamos por analizar cuántos no poseen remuneración.

In [9]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'].isin([5, 6, 7, 8])) & (df['cluster'] == cluster)].copy()
    p_rem_H = df_slice[df_slice['sexo'] == 'H']['remuneracion_c_remunerado'].sum() / df_slice[df_slice['sexo'] == 'H']['poblacion_total'].sum()
    p_rem_M = df_slice[df_slice['sexo'] == 'M']['remuneracion_c_remunerado'].sum() / df_slice[df_slice['sexo'] == 'M']['poblacion_total'].sum()
    print('La cantidad de hombres de edad 20-39 que tienen trabajo remunerado es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'H']['remuneracion_c_remunerado'].sum(), p_rem_H * 100))
    print('La cantidad de hombres de edad 20-39 que tienen trabajo remunerado es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'M']['remuneracion_c_remunerado'].sum(), p_rem_M * 100))
    # Estadística
    crosstab_counts = df_slice.groupby('sexo')[['remuneracion_c_remunerado', 'remuneracion_c_no_remunerado']].sum()
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Remuneración {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 3
La cantidad de hombres de edad 20-39 que tienen trabajo remunerado es 3071703~75.03%
La cantidad de hombres de edad 20-39 que tienen trabajo remunerado es 2490496~58.96%
Remuneración no es independiente de sexo

Para el cluster 2
La cantidad de hombres de edad 20-39 que tienen trabajo remunerado es 60681~44.28%
La cantidad de hombres de edad 20-39 que tienen trabajo remunerado es 28464~21.43%
Remuneración no es independiente de sexo



## Escena 7
Edad 5 (25 a 39) Análisis migratorio

In [10]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'].isin([5, 6, 7, 8])) & (df['cluster'] == cluster)].copy()
    p_inm_H = df_slice[df_slice['sexo'] == 'H']['inmigracion5_c_si'].sum() / df_slice[df_slice['sexo'] == 'H']['poblacion_total'].sum()
    p_inm_M = df_slice[df_slice['sexo'] == 'M']['inmigracion5_c_si'].sum() / df_slice[df_slice['sexo'] == 'M']['poblacion_total'].sum()
    print('La cantidad de hombres de edad 20-39 que debieron inmigrar es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'H']['inmigracion5_c_si'].sum(), p_inm_H * 100))
    print('La cantidad de mujeres de edad 20-39 que debieron inmigrar es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'M']['inmigracion5_c_si'].sum(), p_inm_M * 100))
    # Estadística
    crosstab_counts = df_slice.groupby('sexo')[['inmigracion5_c_si', 'inmigracion5_c_no']].sum()
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Inmigracion {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 3
La cantidad de hombres de edad 20-39 que debieron inmigrar es 584271~14.27%
La cantidad de mujeres de edad 20-39 que debieron inmigrar es 598078~14.16%
Inmigracion no es independiente de sexo

Para el cluster 2
La cantidad de hombres de edad 20-39 que debieron inmigrar es 7564~5.52%
La cantidad de mujeres de edad 20-39 que debieron inmigrar es 8034~6.05%
Inmigracion no es independiente de sexo



## Escena 8
Edad 5 (40 a 59) Query: Agrupar por estado civil, remunerado y obtener cantidades

In [11]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'].isin([9, 10, 11, 12])) & (df['cluster'] == cluster)].copy()
    p_inm_H = df_slice[df_slice['sexo'] == 'H']['remuneracion_x_pareja_c_no_remunerado_x_si'].sum() / df_slice[df_slice['sexo'] == 'H']['poblacion_total'].sum()
    p_inm_M = df_slice[df_slice['sexo'] == 'M']['remuneracion_x_pareja_c_no_remunerado_x_si'].sum() / df_slice[df_slice['sexo'] == 'M']['poblacion_total'].sum()
    print('La cantidad de hombres de edad 40-59 emparejados no remunerados es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'H']['remuneracion_x_pareja_c_no_remunerado_x_si'].values[0], p_inm_H * 100))
    print('La cantidad de mujeres de edad 40-59 emparejados no remunerados es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'M']['remuneracion_x_pareja_c_no_remunerado_x_si'].values[0], p_inm_M * 100))
    # Estadística
    crosstab_counts = np.array([[df_slice['remuneracion_x_pareja_c_remunerado_x_si'].sum(), df_slice['remuneracion_x_pareja_c_no_remunerado_x_si'].sum()],
         [df_slice['remuneracion_x_pareja_c_remunerado_x_no'].sum(), df_slice['remuneracion_x_pareja_c_no_remunerado_x_no'].sum()]])
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Estado civil {0:s}es independiente de remuneración\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 3
La cantidad de hombres de edad 40-59 emparejados no remunerados es 45208~7.16%
La cantidad de mujeres de edad 40-59 emparejados no remunerados es 211793~28.13%
Estado civil no es independiente de remuneración

Para el cluster 2
La cantidad de hombres de edad 40-59 emparejados no remunerados es 7252~32.41%
La cantidad de mujeres de edad 40-59 emparejados no remunerados es 12255~53.11%
Estado civil es independiente de remuneración



## Escena 9
Edades 40-59. Query: Filtrar por remunerado y ver la cantidad de nivel educativo por sexo.

Las mujeres requieren de mayor competitividad académica para poder ser remuneradas.

In [12]:
for cluster in iterator:
    print('Para el cluster {0:d}'.format(cluster))
    # EDA
    df_slice = df[(df['edad'].isin([9, 10, 11, 12])) & (df['cluster'] == cluster)].copy()
    p_rem_no_H = df_slice[df_slice['sexo'] == 'H']['remuneracion_x_escolaridad_c_remunerado_x_no_escolaridad'].sum() / df_slice[df_slice['sexo'] == 'H']['poblacion_total'].sum()
    p_rem_no_M = df_slice[df_slice['sexo'] == 'M']['remuneracion_x_escolaridad_c_remunerado_x_no_escolaridad'].sum() / df_slice[df_slice['sexo'] == 'M']['poblacion_total'].sum()
    p_rem_bas_H = df_slice[df_slice['sexo'] == 'H']['remuneracion_x_escolaridad_c_remunerado_x_basico'].sum() / df_slice[df_slice['sexo'] == 'H']['poblacion_total'].sum()
    p_rem_bas_M = df_slice[df_slice['sexo'] == 'M']['remuneracion_x_escolaridad_c_remunerado_x_basico'].sum() / df_slice[df_slice['sexo'] == 'M']['poblacion_total'].sum()
    p_rem_av_H = df_slice[df_slice['sexo'] == 'H']['remuneracion_x_escolaridad_c_remunerado_x_avanzado'].sum() / df_slice[df_slice['sexo'] == 'H']['poblacion_total'].sum()
    p_rem_av_M = df_slice[df_slice['sexo'] == 'M']['remuneracion_x_escolaridad_c_remunerado_x_avanzado'].sum() / df_slice[df_slice['sexo'] == 'M']['poblacion_total'].sum()
    print('La cantidad de hombres de edad 40-59 remunerados sin educacion es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'H']['remuneracion_x_escolaridad_c_remunerado_x_no_escolaridad'].sum(), p_rem_no_H * 100))
    print('La cantidad de hombres de edad 40-59 remunerados con educacion basica es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'H']['remuneracion_x_escolaridad_c_remunerado_x_basico'].sum(), p_rem_bas_H * 100))
    print('La cantidad de hombres de edad 40-59 remunerados con educacion avanzada es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'H']['remuneracion_x_escolaridad_c_remunerado_x_avanzado'].sum(), p_rem_av_H * 100))
    print('La cantidad de mujeres de edad 40-59 remunerados sin educacion es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'M']['remuneracion_x_escolaridad_c_remunerado_x_no_escolaridad'].sum(), p_rem_no_M * 100))
    print('La cantidad de mujeres de edad 40-59 remunerados con educacion basica es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'M']['remuneracion_x_escolaridad_c_remunerado_x_basico'].sum(), p_rem_bas_M * 100))
    print('La cantidad de mujeres de edad 40-59 remunerados con educacion avanzada es {0:d}~{1:.2f}%'.
          format(df_slice[df_slice['sexo'] == 'M']['remuneracion_x_escolaridad_c_remunerado_x_avanzado'].sum(), p_rem_av_M * 100))
    # Estadística
    crosstab_counts = df_slice.groupby('sexo')[['remuneracion_x_escolaridad_c_remunerado_x_no_escolaridad', 'remuneracion_x_escolaridad_c_remunerado_x_basico', 'remuneracion_x_escolaridad_c_remunerado_x_avanzado']].sum()
    _, p_val, _, _ = scipy.stats.chi2_contingency(crosstab_counts)
    print('Remuneracion por nivel de escolaridad {0:s}es independiente de sexo\n'.format('no ' if p_val < 0.05 else ''))

Para el cluster 3
La cantidad de hombres de edad 40-59 remunerados sin educacion es 52919~1.88%
La cantidad de hombres de edad 40-59 remunerados con educacion basica es 1643878~58.28%
La cantidad de hombres de edad 40-59 remunerados con educacion avanzada es 705526~25.01%
La cantidad de mujeres de edad 40-59 remunerados sin educacion es 33092~0.99%
La cantidad de mujeres de edad 40-59 remunerados con educacion basica es 1118505~33.41%
La cantidad de mujeres de edad 40-59 remunerados con educacion avanzada es 752375~22.47%
Remuneracion por nivel de escolaridad no es independiente de sexo

Para el cluster 2
La cantidad de hombres de edad 40-59 remunerados sin educacion es 7975~10.97%
La cantidad de hombres de edad 40-59 remunerados con educacion basica es 25133~34.57%
La cantidad de hombres de edad 40-59 remunerados con educacion avanzada es 3231~4.44%
La cantidad de mujeres de edad 40-59 remunerados sin educacion es 3692~5.16%
La cantidad de mujeres de edad 40-59 remunerados con educaci